# Data Preperation

### Goals
- Deal with missing data
- Index dates
- Transform data to nice numpy arrays

In [103]:
import pandas as pd
import datetime

In [104]:
pd.read_csv("raw_data_covid.csv")

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22,3/4/22,3/5/22,3/6/22,3/7/22
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,173146,173395,173659,173879,174073,174214,174214,174331,174582,175000
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,271141,271527,271563,271702,271825,271825,272030,272030,272210,272250
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,264778,264855,264936,265010,265079,265130,265186,265227,265265,265297
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,37999,37999,37999,38165,38249,38342,38434,38434,38434,38620
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,98701,98701,98741,98746,98746,98746,98796,98796,98806,98806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,640214,640214,645947,647203,648039,649110,649971,649971,649971,651700
280,NaN,Winter Olympics 2022,39.904200,116.407400,0,0,0,0,0,0,...,514,518,521,524,524,526,530,530,532,532
281,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,11760,11769,11771,11771,11771,11772,11774,11775,11777,11781
282,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,312611,312707,312750,312970,313203,313394,313613,313744,313821,313910


In [105]:
pd.read_csv("raw_data_omxh.csv")

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-01-22,4365.709961,4377.720215,4343.870117,4352.100098,4352.100098,42163100.0
1,2020-01-23,4350.950195,4359.919922,4303.509766,4324.770020,4324.770020,41634900.0
2,2020-01-24,4323.339844,4370.490234,4323.339844,4344.259766,4344.259766,42872400.0
3,2020-01-27,4327.450195,4327.450195,4240.470215,4246.680176,4246.680176,49010300.0
4,2020-01-28,4251.250000,4336.950195,4234.939941,4323.930176,4323.930176,46380700.0
...,...,...,...,...,...,...,...
521,2022-02-14,5155.339844,5166.680176,5082.790039,5146.870117,5146.870117,0.0
522,2022-02-15,5136.410156,5221.729980,5126.850098,5209.259766,5209.259766,0.0
523,2022-02-16,5235.709961,5257.430176,5204.839844,5229.529785,5229.529785,43621700.0
524,2022-02-17,5229.540039,5236.060059,5137.810059,5151.890137,5151.890137,0.0


In [106]:
# Read the files
df_covid = pd.read_csv("not_so_raw_data_covid.csv") #The files has been prepared a little with Google Sheets (Deleted other region cases, Transposed erc.)
df_omxh = pd.read_csv("raw_data_omxh.csv").drop(columns=["Open", "High", "Low", "Adj Close", "Volume"])


# Format the dates
df_covid = df_covid.apply(lambda x: [datetime.datetime.strptime(x[0],"%m/%d/%y").strftime("%Y-%m-%d"), x[1]] ,axis=1, result_type="expand")
df_covid = df_covid.rename(columns={0 : "Date", 1 : "Cases"})

# Join the data frames
df = df_covid.join(df_omxh.set_index("Date"), on="Date")

df.head(20)

,Date,Cases,Close
0,2020-01-22,0,4352.100098
1,2020-01-23,0,4324.770020
2,2020-01-24,0,4344.259766
3,2020-01-25,0,NaN
4,2020-01-26,0,NaN
5,2020-01-27,0,4246.680176
6,2020-01-28,0,4323.930176
7,2020-01-29,1,4315.410156
8,2020-01-30,1,4322.740234
9,2020-01-31,1,4300.080078


In [107]:
df.tail(20)

,Date,Cases,Close
756,2022-02-16,592765,5229.529785
757,2022-02-17,600718,5151.890137
758,2022-02-18,605046,5115.390137
759,2022-02-19,605046,NaN
760,2022-02-20,605046,NaN
761,2022-02-21,622717,NaN
762,2022-02-22,625463,NaN
763,2022-02-23,629727,NaN
764,2022-02-24,637658,NaN
765,2022-02-25,637729,NaN


In [108]:
# Clear the points with missing data
df = df.dropna()

# Reset the index so we can use the indexes as time
df = df.reset_index()

# Drop the Date culumn
df = df.drop(columns="Date")

df

,index,Cases,Close
0,0,0,4352.100098
1,1,0,4324.770020
2,2,0,4344.259766
3,5,0,4246.680176
4,6,0,4323.930176
...,...,...,...
518,754,582383,5146.870117
519,755,586809,5209.259766
520,756,592765,5229.529785
521,757,600718,5151.890137


In [109]:
# Get the next week Close value and join it
next_wk_close = df.apply(lambda x: [ x[0] - 7, x[1], x[2] ], axis=1, result_type="expand")

# Join the data frames, drop old cases, drop missing, rename columns, drop last duplicates
df = df.join(next_wk_close.set_index(0), on="index").drop(columns=1).fillna(method="pad").rename(columns={"index" : "Time", 2 : "next_Close"}).iloc[:-5]

df

,Time,Cases,Close,next_Close
0,0,0,4352.100098,4315.410156
1,1,0,4324.770020,4322.740234
2,2,0,4344.259766,4300.080078
3,5,0,4246.680176,4294.100098
4,6,0,4323.930176,4359.759766
...,...,...,...,...
513,747,534790,5299.080078,5146.870117
514,748,542262,5287.819824,5209.259766
515,749,546782,5339.259766,5229.529785
516,750,559587,5297.140137,5151.890137


In [110]:
# Export to .csv file
df.to_csv("cleaned_data.csv", index=False)